In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
year = str(2019)

In [3]:
results = pd.read_csv(f'datasets/{year}/cpl-{year}-results.csv')
stats = pd.read_csv('datasets/2019/cpl-2019-stats.csv')

In [4]:
teams = results['home'].unique()
teams = np.sort(teams,axis=-1)

In [5]:
if year == '2019':
    teams_short = ['CFC', 'FCE', 'FFC', 'HFX', 'PFC', 'VFC', 'Y9']
else:
    teams_short = ['AO','CFC', 'FCE', 'FFC', 'HFX', 'PFC', 'VFC', 'Y9']

In [6]:
a =[]
for team in teams:
    a.append(team)
teams = a

In [7]:
teams

['Cavalry FC',
 'FC Edmonton',
 'Forge FC',
 'HFX Wanderers FC',
 'Pacific FC',
 'Valour FC',
 'York9 FC']

In [8]:
team_ref = pd.DataFrame({'team': teams, 'short': teams_short})

In [9]:
team_ref

,team,short
0,Cavalry FC,CFC
1,FC Edmonton,FCE
2,Forge FC,FFC
3,HFX Wanderers FC,HFX
4,Pacific FC,PFC
5,Valour FC,VFC
6,York9 FC,Y9


In [10]:
results.head(5)

,game,s,d,m,hs,as,home,hr,away,ar,csh,csa,combined,venue,links
0,I1,0,27,4,1,1,Forge FC,D,York9 FC,D,0,0,4-27-2019 Forge FC D 1-1 D York9 FC,Tim Hortons Field,https://canpl.ca/matchcentre/4ilfbdmlp4zuj7k3c...
1,I2,0,28,4,1,0,Pacific FC,W,HFX Wanderers FC,L,1,0,4-28-2019 Pacific FC W 1-0 L HFX Wanderers FC,Westhills Stadium,https://canpl.ca/matchcentre/4itgc6bq5l5c7iv5k...
2,I3,0,1,5,1,2,Pacific FC,L,Valour FC,W,0,0,5-1-2019 Pacific FC L 1-2 W Valour FC,Westhills Stadium,https://canpl.ca/matchcentre/4j6hzym2ji5zgrm0w...
3,I4,0,4,5,2,1,HFX Wanderers FC,W,Forge FC,L,0,0,5-4-2019 HFX Wanderers FC W 2-1 L Forge FC,Wanderers Grounds,https://canpl.ca/matchcentre/4jopa68wp2k7cntkj...
4,I5,0,4,5,2,1,Cavalry FC,W,York9 FC,L,0,0,5-4-2019 Cavalry FC W 2-1 L York9 FC,ATCO Field,https://canpl.ca/matchcentre/4kj2hnlgvv3ncwouf...


In [11]:
def index_reset(data):
    data = data.reset_index()
    data.pop('index')
    return data

def get_team_results(data,query):
    db = data[data['home'] == query]
    da = data[data['away'] == query]
    db = pd.concat([db,da])
    db = index_reset(db)
    return db

def get_team_brief(data,query):
    db = get_team_results(data,query)
    cols = ['game','s','csh','csa','combined','venue','links']
    for col in cols:
        db.pop(col)
    db = db.sort_values(by=['m','d'])
    db = index_reset(db)
    db['summary'] = '0'
    for i in range(0,db.shape[0]):
        if db.iloc[i]['home'] == query:
            x = team_ref[team_ref['team'] == db.iloc[i]['away']].reset_index()
            opponent = x.iloc[0]['short']
            outcome = db.iloc[i]['hr'] + ' A'
        else:
            x = team_ref[team_ref['team'] == db.iloc[i]['home']].reset_index()
            opponent = x.iloc[0]['short']
            outcome = db.iloc[i]['ar'] + ' A'
        score = str(db.iloc[i]['hs']) + ' - ' + str(db.iloc[i]['as'])
        db.loc[i,'summary'] = outcome + ' ' + score +  ' ' + opponent
    db['team'] = query
    return db

def get_results_brief(data):
    db = pd.DataFrame()
    for team in teams:
        df = get_team_brief(data,team)
        db = pd.concat([db,df])
    db = index_reset(db)
    return db

def get_club_statistics(data,query):
    df = data
    a = []
    cols = df.columns
    for row in range(0,df.shape[0]):
        if df.iloc[row]['home'] == query:
            if df.iloc[row]['hr'] == 'W':
                vals = [1,3,1,0,0,df.iloc[row]['hs'],df.iloc[row]['as'],df.iloc[row]['hs'],df.iloc[row]['as'],0,0]
                a.append(vals)
            if df.iloc[row]['hr'] == 'L':
                vals = [1,0,0,1,0,df.iloc[row]['hs'],df.iloc[row]['as'],df.iloc[row]['hs'],df.iloc[row]['as'],0,0]
                a.append(vals)
            if df.iloc[row]['hr'] == 'D':
                vals = [1,1,0,0,1,df.iloc[row]['hs'],df.iloc[row]['as'],df.iloc[row]['hs'],df.iloc[row]['as'],0,0]
                a.append(vals)
        if df.iloc[row]['away'] == query:
            if df.iloc[row]['ar'] == 'W':
                vals = [1,3,1,0,0,df.iloc[row]['as'],df.iloc[row]['hs'],0,0,df.iloc[row]['as'],df.iloc[row]['hs']]
                a.append(vals)
            if df.iloc[row]['ar'] == 'L':
                vals = [1,0,0,1,0,df.iloc[row]['as'],df.iloc[row]['hs'],0,0,df.iloc[row]['as'],df.iloc[row]['hs']]
                a.append(vals)
            if df.iloc[row]['ar'] == 'D':
                vals = [1,1,0,0,1,df.iloc[row]['as'],df.iloc[row]['hs'],0,0,df.iloc[row]['as'],df.iloc[row]['hs']]
                a.append(vals)
    db= pd.DataFrame(a,columns=['gp','pts','w','l','d','gf','ga','gfh','gah','gfa','gaa'])
    db = pd.DataFrame(db.sum())
    db = db.T
    return db

def get_standings(data,season):
    db = pd.DataFrame()
    if season == 1:
        data = data[data['s'] <= 1]
    if season == 2:
        data = data[data['s'] > 1]
    teams = data['home'].unique()
    teams = np.sort(teams,axis=-1)
    for team in teams:
        df = get_team_brief(data,team)
        df = get_club_statistics(df,team)
        df.insert(0,'team',team)
        db = pd.concat([db,df])
    db = db.sort_values(by=['pts','w','gf'],ascending=False)
    db = index_reset(db)
    return db

In [12]:
query = 'HFX Wanderers FC'

In [13]:
team = get_club_statistics(results,query)

In [14]:
team

,gp,pts,w,l,d,gf,ga,gfh,gah,gfa,gaa
0,28,28,6,12,10,21,35,13,9,8,26


In [15]:
team_standings = get_standings(results,1)

In [16]:
team_standings

,team,gp,pts,w,l,d,gf,ga,gfh,gah,gfa,gaa
0,Cavalry FC,28,62,19,4,5,51,19,26,8,25,11
1,Forge FC,28,56,17,6,5,45,26,25,10,20,16
2,York9 FC,28,34,9,12,7,39,37,18,19,21,18
3,FC Edmonton,28,32,8,12,8,27,33,15,14,12,19
4,Pacific FC,28,31,8,13,7,35,46,22,20,13,26
5,Valour FC,28,28,8,16,4,30,52,17,32,13,20
6,HFX Wanderers FC,28,28,6,12,10,21,35,13,9,8,26


In [17]:
championship = get_standings(results,2)
#championship = championship[championship['gp'] > 1]
championship

,team,gp,pts,w,l,d,gf,ga,gfh,gah,gfa,gaa
0,Forge FC,2,6,2,0,0,2,0,1,0,1,0
1,Cavalry FC,2,0,0,2,0,0,2,0,1,0,1


In [18]:
championship.to_csv(f'datasets/{year}/cpl-{year}-championship.csv',index=False)

In [19]:
#team_standings.to_csv(f'datasets/{year}/cpl-{year}-team_standings.csv',index=False)

In [20]:
results_brief = get_results_brief(results)

In [21]:
results_brief.head(5)

,d,m,hs,as,home,hr,away,ar,summary,team
0,4,5,2,1,Cavalry FC,W,York9 FC,L,W A 2 - 1 Y9,Cavalry FC
1,8,5,1,0,Cavalry FC,W,Valour FC,L,W A 1 - 0 VFC,Cavalry FC
2,12,5,1,2,Forge FC,L,Cavalry FC,W,W A 1 - 2 FFC,Cavalry FC
3,18,5,1,0,Cavalry FC,W,FC Edmonton,L,W A 1 - 0 FCE,Cavalry FC
4,25,5,2,0,Cavalry FC,W,HFX Wanderers FC,L,W A 2 - 0 HFX,Cavalry FC


In [22]:
#results_brief.to_csv(f'datasets/{year}/cpl-{year}-results_brief.csv',index=False)

In [23]:
stats.head(5)

,game,team,position,number,name,first,last,minutes,touches,passes,...,clearances,interceptions,yellow,red,fouls won,fouls conceded,shots faced,saves,claimed crosses,clean sheets
0,I1,Forge FC,m,1,Alexander Achinioti-Jönsson,Alexander,Achinioti-Jönsson,77.0,47.0,41.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,I1,Forge FC,d,16,Bertrand Owundi,Bertrand,Owundi,77.0,69.0,56.0,...,5.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,I1,Forge FC,f,13,Christopher Nanco,Christopher,Nanco,56.0,42.0,27.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,I1,Forge FC,d,22,Dominic Samuel,Dominic,Samuel,90.0,59.0,48.0,...,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,I1,Forge FC,m,6,Elimane Cissé,Elimane,Cissé,90.0,75.0,61.0,...,1.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [24]:
stats.describe()

,number,minutes,touches,passes,pass accuracy,crosses,cross accuracy,assists,chances,goals,...,clearances,interceptions,yellow,red,fouls won,fouls conceded,shots faced,saves,claimed crosses,clean sheets
count,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000,...,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000,2735.000000
mean,13.829250,71.540768,44.576965,30.244241,0.736570,1.116271,0.093119,0.060695,0.628154,0.087751,...,1.210238,0.790128,0.112249,0.006216,0.938574,0.995978,0.318099,0.227422,0.036563,0.019744
std,11.355872,28.545983,23.277512,18.645196,0.233204,1.904354,0.235873,0.246352,0.989120,0.311297,...,1.859761,1.131050,0.315730,0.078609,1.119698,1.125445,1.295603,0.956352,0.251072,0.139145
min,1.000000,1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,61.000000,28.000000,16.000000,0.670000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,12.000000,90.000000,45.000000,29.000000,0.780000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,19.000000,90.000000,61.000000,42.000000,0.850000,2.000000,0.000000,0.000000,1.000000,0.000000,...,2.000000,1.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000
max,92.000000,90.000000,118.000000,112.000000,1.000000,15.000000,1.000000,2.000000,7.000000,3.000000,...,11.000000,7.000000,1.000000,1.000000,6.000000,7.000000,13.000000,10.000000,4.000000,1.000000


In [25]:
def get_team_stats(data,query):
    db = data[data['team'] == query]
    names = db['name'].unique()
    information = data.copy()
    db.pop('number')
    db = db.groupby(['name']).sum()
    db.insert(0,'last','empty')
    db.insert(0,'first','empty')
    db.insert(0,'position','empty')
    db.insert(0,'number',0)
    #db.insert(0,'team',team)
    i = 0
    for name in names:
        player = information[information['name'] == name].head(1)
        db.at[name,'first'] = player.iloc[0]['first']
        db.at[name,'last'] = player.iloc[0]['last']
        db.at[name,'number'] = int(player.iloc[0]['number'])
        db.at[name,'position'] = player.iloc[0]['position']
        db.at[name,'pass accuracy'] = player.iloc[0]['pass accuracy'].mean()
        db.at[name,'cross accuracy'] = player.iloc[0]['cross accuracy'].mean()
    db = db.reset_index()
    return db

def get_stats_all(data):
    db = pd.DataFrame()
    for team in teams:
        df = get_team_stats(data,team)
        df.insert(0,'team',team)
        db = pd.concat([db,df])
    db = index_reset(db)
    return db

In [26]:
team_stats = get_stats_all(stats)#,query)

In [27]:
team_stats.head(5)

,team,name,number,position,first,last,minutes,touches,passes,pass accuracy,...,clearances,interceptions,yellow,red,fouls won,fouls conceded,shots faced,saves,claimed crosses,clean sheets
0,Cavalry FC,Aribim Pepple,24,f,Aribim,Pepple,60.0,21.0,9.0,1.00,...,0.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0
1,Cavalry FC,Carlos Patiño,20,m,Carlos,Patiño,320.0,238.0,122.0,0.88,...,1.0,5.0,0.0,0.0,13.0,12.0,0.0,0.0,0.0,0.0
2,Cavalry FC,Dean Northover,12,d,Dean,Northover,609.0,478.0,255.0,0.67,...,10.0,15.0,2.0,1.0,14.0,13.0,0.0,0.0,0.0,0.0
3,Cavalry FC,Dominick Zator,4,d,Dominick,Zator,2335.0,1866.0,1402.0,0.83,...,75.0,51.0,0.0,0.0,19.0,15.0,0.0,0.0,0.0,0.0
4,Cavalry FC,Dominique Malonga,19,f,Dominique,Malonga,1871.0,760.0,489.0,0.80,...,12.0,2.0,1.0,0.0,31.0,8.0,0.0,0.0,0.0,0.0


In [28]:
#team_stats.to_csv(f'datasets/{year}/cpl-{year}-stats_combined.csv',index=False)

In [46]:
# get associated information for players league wide and calculate an overall score for each position
def get_evaluation(db,df):
    names = db.name.unique() # grab the list of names at the specified position
    eval_ = db.describe().T # get the evalution scores
    checks = db.columns[3:] # slice away the first three columns (name,number,postion) not needed
    db['overall'] = 0.0 # create the final column overall
    db = db.set_index('name') # set the index to the player name to search for a specific player
    for name in names: # iterate through the names in the lisst
        player = df[df['name'] == name].head(1) # get the players details
        a = [] # create an empty array to store the scores
        for check in checks: # iterate through the columns of remaining data
            result = player.iloc[0][check] / eval_['max'][check] # calculate the score for the value found value/max
            a.append(result) # append the result into the list
            overall = round(sum(a) / len(checks),2) #calculate the final score sum(list) / num of checks
            db.at[name,'overall'] = overall # assign the value as the overall score
    db = db.reset_index() # reset the index, making the name column a column again
    db = db.sort_values(by=['overall'],ascending=False) # sort using overall, descending
    return db

def top_goalscorers(data):
    df = data.copy()
    cols = ['name','number','position','minutes','goals']
    db = df[cols]
    db = db.sort_values(by=['goals'],ascending=False)
    db = db.reset_index()
    return db

def top_forwards(data): # get the forwards in the league
    player_information = data.copy() # load player information
    cols = ['name','number','position','minutes','goals','chances','assists','shots','shots on target','passes','crosses','duels','tackles']
    df = player_information[player_information['position'] == 'f'] # get the forwards where position = f
    db = df[cols] # select specific columns associated with the evaluation
    db = get_evaluation(db,df) 
    db = index_reset(db)
    names = db.name.unique() # get the names of the players who fit the criteria
    db = db.set_index('name') # set the index to the name column to make the search possible
    for name in names:
        player = df[df['name'] == name].head(1) # forwards main purpose is to score goals
        '''if (player.iloc[0]['goals'] <= 2.0 and player.iloc[0]['minutes'] >= 1000.0): # if player scores less than 2 & has minutes greater than 1000
            db.at[name,'overall'] = db.at[name,'overall'] - 0.2
        if player.iloc[0]['goals'] >= 8.0: # reward forwards scoring greater than 8 goals
            db.at[name,'overall'] = db.at[name,'overall'] + 0.1'''
    db = db.sort_values(by=['overall'],ascending=False)
    db = db.reset_index()
    return db

def top_midfielders(data): # get the midfielders in the league
    player_information = data.copy()
    cols = ['name','number','position','minutes','goals','assists','touches','passes','pass accuracy','crosses','cross accuracy','chances','duels','tackles']
    df = player_information[player_information['position'] == 'm'] # get the midfields where position = m
    db = df[cols]
    db = get_evaluation(db,df)
    db = index_reset(db)
    names = db.name.unique()
    db = db.set_index('name')
    for name in names:
        player = df[df['name'] == name].head(1)
        '''if player.iloc[0]['goals'] <= 2.0:
            db.at[name,'overall'] = db.at[name,'overall'] - 0.2
        if player.iloc[0]['goals'] >= 8.0:
            db.at[name,'overall'] = db.at[name,'overall'] + 0.1'''
    db = db.sort_values(by=['overall'],ascending=False)
    db = db.reset_index()
    return db

def top_defenders(data):  # get the defenders in the league
    player_information = data.copy()
    cols = ['name','number','position','minutes','tackles','tackles won','clearances','interceptions','duels','duels won']
    df = player_information[player_information['position'] == 'd'] # get the defenders where position = d
    db = df[cols]
    db = get_evaluation(db,df)
    db = index_reset(db)
    return db

def top_keepers(data):  # get the keepers in the league
    player_information = data.copy()
    cols = ['name','number','position','minutes','clean sheets','saves','shots faced','claimed crosses']
    df = player_information[player_information['position'] == 'g'] # get the goalkeepers where position = g
    db = df[cols]
    db = get_evaluation(db,df)
    db = index_reset(db)
    return db

def top_offenders(data):  # get the offences handed out in the league
    player_information = data.copy()
    cols = ['name','number','position','minutes','yellow','red','fouls conceded']
    df = player_information
    db = df[cols]
    db = get_evaluation(db,df)
    db = db.sort_values(by=['red','yellow'],ascending=False)
    db = db.reset_index()
    db.pop('index')
    return db

In [47]:
rated_forwards = top_forwards(team_stats)
rated_midfielders = top_midfielders(team_stats)
rated_defenders = top_defenders(team_stats)
rated_keepers = top_keepers(team_stats)
rated_offenders = top_offenders(team_stats)
rated_goalscorers = top_goalscorers(team_stats)

/home/todd/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [41]:
#rated_forwards.to_csv(f'datasets/{year}/cpl-{year}-forwards.csv',index=False)
#rated_midfielders.to_csv(f'datasets/{year}/cpl-{year}-midfielders.csv',index=False)
#rated_defenders.to_csv(f'datasets/{year}/cpl-{year}-defenders.csv',index=False)
#rated_keepers.to_csv(f'datasets/{year}/cpl-{year}-keepers.csv',index=False)
#rated_offenders.to_csv(f'datasets/{year}/cpl-{year}-discipline.csv',index=False)

In [48]:
rated_goalscorers.head(10)

,index,name,number,position,minutes,goals
0,69,Tristan Borges,4,m,1884.0,13.0
1,4,Dominique Malonga,19,f,1871.0,11.0
2,113,Terran Campbell,14,f,2334.0,11.0
3,32,Easton Ongaro,19,f,1167.0,10.0
4,157,Rodrigo Gattas,22,f,1958.0,9.0
5,159,Ryan Telfer,18,m,2216.0,8.0
6,20,Oliver Minatel,7,f,1450.0,7.0
7,128,Marco Bustos,22,m,2249.0,7.0
8,160,Simon Karlsson Adjei,12,f,1422.0,7.0
9,71,Akeem Garcia,11,m,2082.0,7.0


In [37]:
rated_forwards.head(10)

,name,number,position,minutes,goals,chances,assists,shots,shots on target,passes,crosses,duels,tackles,overall
0,Mohamed Kourouma,12,f,1988.0,1.0,44.0,3.0,68.0,17.0,702.0,156.0,264.0,56.0,0.77
1,Terran Campbell,14,f,2334.0,11.0,31.0,2.0,53.0,23.0,569.0,29.0,324.0,39.0,0.70
2,Nico Pasquotti,17,f,1853.0,5.0,24.0,3.0,33.0,16.0,457.0,61.0,374.0,47.0,0.64
3,Rodrigo Gattas,22,f,1958.0,9.0,23.0,0.0,77.0,36.0,728.0,38.0,214.0,17.0,0.61
4,Oumar Diouck,45,f,1977.0,6.0,30.0,2.0,45.0,16.0,583.0,53.0,269.0,36.0,0.61
5,Dominique Malonga,19,f,1871.0,11.0,19.0,1.0,68.0,34.0,489.0,5.0,185.0,2.0,0.55
6,Simon Karlsson Adjei,12,f,1422.0,7.0,21.0,2.0,53.0,16.0,341.0,4.0,210.0,15.0,0.47
7,Oliver Minatel,7,f,1450.0,7.0,17.0,1.0,37.0,12.0,348.0,21.0,194.0,29.0,0.43
8,Easton Ongaro,19,f,1167.0,10.0,11.0,2.0,38.0,20.0,249.0,6.0,188.0,5.0,0.43
9,Christopher Nanco,13,f,1429.0,4.0,21.0,2.0,29.0,12.0,380.0,22.0,208.0,19.0,0.43


In [38]:
rated_midfielders.head(10)

,name,number,position,minutes,goals,assists,touches,passes,pass accuracy,crosses,cross accuracy,chances,duels,tackles,overall
0,Tristan Borges,4,m,1884.0,13.0,5.0,1307.0,680.0,0.74,77.0,0.00,42.0,356.0,59.0,0.71
1,Ryan Telfer,18,m,2216.0,8.0,1.0,1533.0,832.0,0.60,164.0,0.00,66.0,372.0,52.0,0.70
2,Julian Büscher,8,m,1678.0,5.0,4.0,1432.0,971.0,0.80,113.0,0.29,38.0,282.0,57.0,0.68
3,Kyle Bekker,3,m,2354.0,4.0,5.0,1748.0,1362.0,0.82,98.0,0.25,44.0,173.0,23.0,0.68
4,Manuel Aparicio,10,m,2159.0,2.0,2.0,1788.0,1273.0,0.67,40.0,0.33,37.0,353.0,56.0,0.64
5,Marco Bustos,22,m,2249.0,7.0,3.0,1417.0,912.0,0.93,74.0,0.00,43.0,299.0,22.0,0.61
6,Louis Béland-Goyette,5,m,1772.0,1.0,3.0,1160.0,882.0,0.65,10.0,0.67,28.0,217.0,54.0,0.55
7,Kyle Porter,19,m,1916.0,2.0,4.0,1131.0,729.0,0.76,88.0,0.00,31.0,216.0,47.0,0.55
8,José Escalante,11,m,1592.0,4.0,4.0,905.0,450.0,0.68,116.0,0.25,29.0,226.0,32.0,0.53
9,Victor Blasco,19,m,1586.0,6.0,3.0,1040.0,589.0,0.67,47.0,0.00,27.0,349.0,43.0,0.52


In [37]:
rated_defenders.head(10)

,name,number,position,minutes,tackles,tackles won,clearances,interceptions,duels,duels won,overall
0,Mélé Temguia,8,d,2430.0,61.0,41.0,82.0,51.0,244.0,140.0,0.87
1,André Bona,3,d,2053.0,66.0,50.0,76.0,32.0,257.0,150.0,0.84
2,Luca Gasparotto,13,d,2520.0,31.0,24.0,117.0,49.0,254.0,182.0,0.83
3,Dominick Zator,4,d,2335.0,46.0,32.0,75.0,51.0,265.0,169.0,0.83
4,Diyaeddine Abzi,20,d,1814.0,48.0,35.0,31.0,48.0,267.0,134.0,0.72
5,Kwame Awuah,2,d,1885.0,57.0,40.0,17.0,39.0,258.0,124.0,0.71
6,Daniel Krutzen,10,d,2381.0,48.0,24.0,85.0,42.0,186.0,110.0,0.71
7,Kadin Chung,7,d,2032.0,57.0,42.0,42.0,31.0,203.0,111.0,0.69
8,Matthew Arnone,23,d,2094.0,31.0,20.0,99.0,49.0,170.0,106.0,0.67
9,Ndzemdzela Langwa,20,d,1281.0,65.0,47.0,20.0,14.0,272.0,154.0,0.67


In [38]:
rated_keepers.head(10)

,name,number,position,minutes,clean sheets,saves,shots faced,claimed crosses,overall
0,Nathan Ingham,1,g,2340.0,6.0,88.0,122.0,17.0,0.93
1,Marco Carducci,1,g,2160.0,9.0,65.0,81.0,11.0,0.79
2,Triston Henry,9,g,1890.0,8.0,53.0,70.0,12.0,0.72
3,Connor James,1,g,1980.0,6.0,67.0,91.0,9.0,0.71
4,Mark Village,1,g,1440.0,4.0,47.0,69.0,10.0,0.55
5,Tyson Farago,1,g,1440.0,2.0,47.0,77.0,13.0,0.55
6,Christian Oxner,50,g,1489.0,5.0,41.0,56.0,10.0,0.54
7,Nolan Wirth,6,g,1080.0,1.0,59.0,83.0,6.0,0.46
8,Mathias Janssens,26,g,1080.0,4.0,43.0,65.0,0.0,0.39
9,Jan-Michael Williams,21,g,1028.0,2.0,40.0,59.0,4.0,0.37


In [39]:
rated_offenders.head(20)

,name,number,position,minutes,yellow,red,fouls conceded,overall
0,Manuel Aparicio,10,m,2159.0,5.0,1.0,56.0,0.83
1,Dominic Samuel,22,d,2044.0,5.0,1.0,38.0,0.74
2,Matthew Baldisimo,17,m,1809.0,5.0,1.0,34.0,0.71
3,Alexander González,28,m,1746.0,4.0,1.0,34.0,0.67
4,Jordan Murrell,4,d,1784.0,4.0,1.0,30.0,0.66
5,Joel Waterman,15,m,1719.0,4.0,1.0,20.0,0.61
6,Adam Mitter,15,d,1513.0,4.0,1.0,23.0,0.60
7,Nathan Mavila,3,d,1952.0,2.0,1.0,38.0,0.65
8,Tristan Borges,4,m,1884.0,2.0,1.0,37.0,0.64
9,Lukas MacNaughton,14,d,1704.0,2.0,1.0,24.0,0.57


In [40]:
def get_match_tables(data,query):
    db = data[data['home'] == query]
    db = pd.concat([db,data[data['away'] == query]])
    db = db.sort_values(by=['m','d'])
    return db

def likelihood_table(data,query):
    df = get_match_tables(data,query)
    a = []
    cols = compare.columns
    for row in range(0,df.shape[0]):
        if df.iloc[row]['home'] == query:
            if df.iloc[row]['hr'] == 'W':
                b = [1,2,1]
                c = [1,0,0]
                d = [1,1,0]
                a.append(b)
                a.append(c)
                a.append(d)
            if df.iloc[row]['hr'] == 'L':
                b = [1,2,0]
                c = [1,0,1]
                d = [1,1,0]
                a.append(b)
                a.append(c)
                a.append(d)
            if df.iloc[row]['hr'] == 'D':
                b = [1,2,0]
                c = [1,0,0]
                d = [1,1,1]
                a.append(b)
                a.append(c)
                a.append(d)
        if df.iloc[row]['away'] == query:
            if df.iloc[row]['ar'] == 'W':
                b = [2,2,1]
                c = [2,0,0]
                d = [2,1,0]
                a.append(b)
                a.append(c)
                a.append(d)
            if df.iloc[row]['ar'] == 'L':
                b = [2,2,0]
                c = [2,0,1]
                d = [2,1,0]
                a.append(b)
                a.append(c)
                a.append(d)
            if df.iloc[row]['ar'] == 'D':
                b = [2,2,0]
                c = [2,0,0]
                d = [2,1,1]
                a.append(b)
                a.append(c)
                a.append(d)
    db= pd.DataFrame(a,columns=['h/a','w/l/d','y/n'])
    return db

def get_team_comparison(data,q1,q2):
    # getting games with q1 in both home or away
    db = data[data['team'] == q1]
    db = db.reset_index()
    db.pop('index')
    # filering down more to get only the games against q2
    db = db.sort_values(by=['m','d'])
    db = db[(db['home'] == q2) | (db['away'] == q2)]
    db = db.reset_index()
    db.pop('index')
    return db

def get_NB_data(data,query):
    db = likelihood_table(data,query)
    dy = db.pop('y/n').to_list()
    dx = [tuple(x) for x in db.values]
    return dx, dy   

In [55]:
# home side
q1 = 'Forge FC'
# away side
q2 = 'Cavalry FC'

In [56]:
compare = get_team_comparison(results_brief,q1,q2)

In [57]:
compare

,d,m,hs,as,home,hr,away,ar,summary,team
0,25,5,0,1,Cavalry FC,L,Forge FC,W,W A 0 - 1 CFC,Forge FC


In [58]:
t1_x, t1_y = get_NB_data(compare,q1)

t2_x, t2_y = get_NB_data(compare,q2)

### Game Prediction

use Gaussian Naive Bayes model to predict the outcome of a home match.

In [59]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB,BernoulliNB
import statistics

In [60]:
def get_gnb_prediction(query,x,y,result):
    
    gnb = GaussianNB()
    bnb = BernoulliNB()
    # Train the model using the training sets
    
    gnb.fit(x,y)
    bnb.fit(x,y)
    
    # use below instead of predicted = model.predict([result]) because we want the probability
    gnb_pred = np.round(gnb.predict_proba([result])[:, 1],decimals=2)
    bnb_pred = np.round(bnb.predict_proba([result])[:, 1],decimals=2)
    
    pred = round((gnb_pred[0] + bnb_pred[0]) / 2,2)
    #print(gnb_pred[0], bnb_pred[0], pred)
    
    return pred

def get_match_prediction_home(query,x,y):
    home_win = get_gnb_prediction(query,x,y,[1,2])
    draw = get_gnb_prediction(query,x,y,[1,1])
    return home_win, draw
    
def get_match_prediction_away(query,x,y):
    away_win = get_gnb_prediction(query,x,y,[2,2])
    return away_win

def get_match_prediction(q1,q2,x1,y1,x2,y2):
    home_win, draw = get_match_prediction_home(q1,x1,y1)
    away_win = get_match_prediction_away(q2,x2,y2)
    return home_win, draw, away_win

In [61]:
home_win, draw, away_win = get_match_prediction(q1,q2,t1_x,t1_y,t2_x,t2_y)

In [62]:
print(q1,'\nwin probability: ', home_win)

Forge FC 
win probability:  0.68


In [63]:
print(q2,'\nwin probability: ', away_win)

Cavalry FC 
win probability:  0.08


In [64]:
print('Draw probability: ', draw)

Draw probability:  0.18


In [65]:
round(home_win + draw + away_win,1)

0.9

In [66]:
for team in teams:
    for t in teams:
        if t == team:
            pass
        else:
            compare = get_team_comparison(results_brief,team,t)
            t1_x, t1_y = get_NB_data(compare,team)
            t2_x, t2_y = get_NB_data(compare,t)
            home_win, draw, away_win = get_match_prediction(team,t,t1_x,t1_y,t2_x,t2_y)
            if round(home_win,1) > round(away_win,1):
                print(team,'WINS against', t,'\n')
            if round(home_win,1) == round(away_win,1):
                print(team,'DRAWS against ', t,'\n')
            if round(away_win,1) > round(home_win,1):
                print(t,' WINS against', team,'\n')

Cavalry FC  WINS against Atletico Ottawa 

Atletico Ottawa DRAWS against  FC Edmonton 

Forge FC  WINS against Atletico Ottawa 

Atletico Ottawa DRAWS against  HFX Wanderers FC 

Pacific FC  WINS against Atletico Ottawa 

Atletico Ottawa DRAWS against  Valour FC 

York9 FC  WINS against Atletico Ottawa 

Cavalry FC WINS against Atletico Ottawa 

Cavalry FC WINS against FC Edmonton 

Forge FC  WINS against Cavalry FC 

Cavalry FC WINS against HFX Wanderers FC 

Cavalry FC WINS against Pacific FC 

Cavalry FC WINS against Valour FC 

Cavalry FC WINS against York9 FC 

FC Edmonton DRAWS against  Atletico Ottawa 

Cavalry FC  WINS against FC Edmonton 

Forge FC  WINS against FC Edmonton 

FC Edmonton WINS against HFX Wanderers FC 

FC Edmonton WINS against Pacific FC 

Valour FC  WINS against FC Edmonton 

York9 FC  WINS against FC Edmonton 

Forge FC WINS against Atletico Ottawa 

Forge FC WINS against Cavalry FC 

Forge FC WINS against FC Edmonton 

Forge FC DRAWS against  HFX Wanderers 

In [255]:
def get_team_form(data,query):
    db = data[data['team'] == query]
    db = pd.DataFrame(db['summary'])
    '''db = db.T
    db.insert(0,'team',query)
    db = db.reset_index()
    db.pop('index')'''
    return db

def get_form_results(data):
    db = pd.DataFrame()
    form = get_results_brief(data[data['s'] <= 1])
    teams = data.home.unique()
    teams = np.sort(teams,axis=-1)
    for team in teams:
        df = get_team_form(form,team)
        db[team] = df['summary'].values
    db = db.T
    db = db.reset_index()
    return db

In [257]:
team_form_results = get_form_results(results)
team_form_results

,index,0,1,2,3,4,5,6,7,8,...,18,19,20,21,22,23,24,25,26,27
0,Cavalry FC,W A 2 - 1 Y9,W A 1 - 0 VFC,W A 1 - 2 FFC,W A 1 - 0 FCE,W A 2 - 0 HFX,W A 0 - 3 FCE,W A 1 - 2 HFX,L A 0 - 1 FFC,W A 0 - 2 Y9,...,D A 1 - 1 PFC,W A 0 - 8 VFC,W A 0 - 1 FCE,W A 4 - 1 PFC,D A 1 - 1 Y9,W A 4 - 1 VFC,W A 2 - 0 HFX,W A 2 - 1 FFC,L A 1 - 0 FFC,W A 3 - 1 FCE
1,FC Edmonton,W A 1 - 2 VFC,D A 0 - 0 PFC,L A 1 - 0 CFC,L A 2 - 0 FFC,L A 0 - 1 VFC,L A 0 - 3 CFC,D A 0 - 0 Y9,W A 1 - 3 PFC,W A 1 - 0 FFC,...,D A 2 - 2 Y9,D A 0 - 0 HFX,L A 0 - 1 CFC,L A 1 - 0 PFC,L A 1 - 3 Y9,D A 1 - 1 HFX,L A 0 - 1 FFC,L A 3 - 1 VFC,W A 3 - 1 PFC,L A 3 - 1 CFC
2,Forge FC,D A 1 - 1 Y9,L A 2 - 1 HFX,W A 3 - 0 PFC,L A 1 - 2 CFC,W A 0 - 2 VFC,W A 0 - 2 Y9,W A 2 - 0 FCE,W A 2 - 1 VFC,W A 0 - 1 CFC,...,W A 2 - 1 Y9,W A 1 - 3 VFC,D A 2 - 2 HFX,D A 1 - 1 HFX,W A 3 - 0 PFC,W A 0 - 1 FCE,W A 1 - 0 Y9,L A 2 - 1 CFC,L A 4 - 0 Y9,W A 1 - 0 CFC
3,HFX Wanderers FC,L A 1 - 0 PFC,W A 2 - 1 FFC,L A 1 - 0 VFC,L A 2 - 0 CFC,D A 1 - 1 Y9,W A 2 - 1 PFC,L A 1 - 2 CFC,D A 0 - 0 Y9,W A 2 - 0 VFC,...,L A 2 - 0 VFC,D A 0 - 0 FCE,D A 2 - 2 FFC,D A 1 - 1 FFC,D A 0 - 0 VFC,D A 1 - 1 FCE,D A 1 - 1 PFC,L A 2 - 0 CFC,D A 1 - 1 PFC,W A 0 - 2 Y9
4,Pacific FC,W A 1 - 0 HFX,L A 1 - 2 VFC,L A 3 - 0 FFC,D A 0 - 0 FCE,D A 2 - 2 Y9,L A 2 - 1 HFX,L A 1 - 0 Y9,W A 1 - 2 VFC,L A 1 - 3 FCE,...,D A 1 - 1 CFC,D A 1 - 1 FFC,L A 0 - 2 Y9,W A 1 - 0 FCE,L A 4 - 1 CFC,L A 3 - 0 FFC,D A 1 - 1 HFX,D A 1 - 1 HFX,L A 3 - 1 FCE,W A 2 - 0 VFC
5,Valour FC,W A 1 - 2 PFC,L A 1 - 2 FCE,L A 1 - 0 CFC,W A 1 - 0 HFX,L A 0 - 2 FFC,W A 0 - 1 FCE,L A 2 - 1 FFC,L A 1 - 2 PFC,L A 2 - 0 HFX,...,L A 2 - 1 PFC,W A 2 - 0 HFX,L A 0 - 8 CFC,L A 1 - 3 FFC,W A 2 - 4 Y9,D A 0 - 0 HFX,L A 4 - 1 CFC,W A 3 - 1 FCE,L A 0 - 4 Y9,L A 2 - 0 PFC
6,York9 FC,D A 1 - 1 FFC,L A 2 - 1 CFC,D A 2 - 2 PFC,L A 0 - 2 FFC,D A 1 - 1 HFX,W A 1 - 0 PFC,D A 0 - 0 FCE,D A 0 - 0 HFX,L A 0 - 2 CFC,...,D A 2 - 2 FCE,L A 2 - 1 FFC,W A 0 - 2 PFC,L A 2 - 4 VFC,W A 1 - 3 FCE,D A 1 - 1 CFC,L A 1 - 0 FFC,W A 4 - 0 FFC,W A 0 - 4 VFC,L A 0 - 2 HFX


In [258]:
team_form_results.to_csv(f'datasets/{year}/cpl-{year}-team_form.csv',index=False)